In [1]:
from sagemaker import get_execution_role
from bs4 import BeautifulSoup 
import pandas as pd
import sagemaker
import boto3
import json
import os

In [2]:
role = get_execution_role()
sess = sagemaker.Session()

In [3]:
bucketName = 'signetringcell'
folderName = 'challenge1-signetringcell-dataset/sig-train-pos/sig-train-pos/'

In [4]:
s3_client = boto3.client('s3')
bucket = s3_client.list_objects(Bucket=bucketName, Prefix=folderName)

In [5]:
import s3fs
fs = s3fs.S3FileSystem()

In [11]:
# Function to create json annotation file from xml file and move that json file to s3
def xml_to_json(data, bucketName, folderName, fileName):
    """
    data: content being read
    bucketName: bucket being read from
    folderName: folder being wrote to
    file_name: xml file being read
    """
    Bs_data = BeautifulSoup(data, "xml") 
    zz=Bs_data.find_all("size")
    #print(zz)
    yyyy=str(zz)
    size=yyyy.partition('\n')[2]
    #print(size)
    wid=size.split("</width>",1)[0] 
    wid=wid.split("<width>",1)[1]
    #print(wid)
    hei=size.split("</height>",1)[0] 
    hei=hei.split("<height>",1)[1]
    #print(hei)
    depth=size.split("</depth>",1)[0] 
    depth=depth.split("<depth>",1)[1]
    #print(depth)
    b_unique = Bs_data.find_all('bndbox') 
    data1={}
   # data1['files']=file_name
# I think we nned to change it to jpeg, i don't know if that would fix it
    data1['file']=fileName.split('/')[-1][:-4] + '.jpeg'
    data1['image_size']=[]
    data1['image_size'].append({
        'width' : wid,
        'height' : hei,
        'depth' : depth
    })
    data1['annotations']=[]
    for i in range(len(b_unique)):
        z=b_unique[i]
        y=str(z)
        yy=y.partition('\n')[2]
        xmin=yy.split("</xmin>",1)[0]
        xmin=xmin.split("<xmin>",1)[1]
        ymin=yy.split("</ymin>",1)[0] 
        ymin=ymin.split("<ymin>",1)[1]
        xmax=yy.split("</xmax>",1)[0] 
        xmax=xmax.split("<xmax>",1)[1]
        ymax=yy.split("</ymax>",1)[0] 
        ymax=ymax.split("<ymax>",1)[1]
        left=int(xmin)
        top=int(ymax)
        height=str(int(ymax)-int(ymin))
        width=str(int(xmax)-int(xmin))
        data1['annotations'].append({
            'class_id': 0,
            'left': left,
            'top' : top,
            'width' : width,
            'height' :height
        })
    data1['categories']=[]
    data1['categories'].append({
        'class_id' : 0,
        'name' : "signet ring cell"
    })
    
    json_fileName = fileName.split('/')[-1][:-4] + '.json'
    with open(json_fileName, 'w') as outfile:
        print("writing json file " + json_fileName)
        json.dump(data1, outfile) 
        
    with open(json_fileName, 'rb') as f:
        print('sending file to s3')
        s3_client.upload_fileobj(f, bucketName, folderName + json_fileName)
        print('file successfully sent')
        
    os.remove(json_fileName)
    

In [12]:
SRC_folderName = 'challenge1-signetringcell-dataset/src_detection_model/train_annotation/'

In [13]:
#Create annotation file for each image
for img_num in range(len(bucket['Contents'])):
    if img_num%2==1:
        file_name = bucket['Contents'][img_num]['Key']
        with fs.open('{}/{}'.format(bucketName, file_name)) as f:
            data = f.read()
            xml_to_json(data, bucketName, SRC_folderName, file_name)

writing json file 2018_64982_1-3_2019-02-25 21_57_36-lv0-33516-59515-2003-2010.json
sending file to s3
file successfully sent
writing json file 2018_64982_1-3_2019-02-25 21_57_36-lv0-34589-61706-2030-2044.json
sending file to s3
file successfully sent
writing json file 2018_64982_1-3_2019-02-25 21_57_36-lv0-36515-58465-2013-2071.json
sending file to s3
file successfully sent
writing json file 2018_64982_1-3_2019-02-25 21_57_36-lv0-37528-60747-2016-2000.json
sending file to s3
file successfully sent
writing json file 2018_64982_1-3_2019-02-25 21_57_36-lv0-38368-62991-2040-2016.json
sending file to s3
file successfully sent
writing json file 2018_64982_1-3_2019-02-25 21_57_36-lv0-39555-58583-2006-2007.json
sending file to s3
file successfully sent
writing json file 2018_64982_1-3_2019-02-25 21_57_36-lv0-39896-60663-2008-2000.json
sending file to s3
file successfully sent
writing json file 2018_64982_1-3_2019-02-25 21_57_36-lv0-47662-55703-2020-2046.json
sending file to s3
file successful